# Venture Funding with Deep Learning

Alphabet Soup is a venture capital firm which receives many funding applications from startups for funding. Our data is a CSV containing information about the organizations that have received funding from Alphabet Soup over the years. A Binary classifier Neural Network model is created to help the company to predict whether an applicant will become a successful business.

* Prepare the data for use on a neural network model.

* Compile and evaluate a binary classification model using a neural network.

* Optimize the neural network model.

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

---

## Prepare the data to be used on a neural network model

Read the `applicants_data.csv` file into a Pandas DataFrame.


In [2]:
# Upload applicants_data.csv to Colab
from google.colab import files

csv_file = files.upload()

Saving applicants_data.csv to applicants_data (1).csv


In [3]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv('applicants_data.csv')

# Review the DataFrame
applicant_data_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Review the data types associated with the columns
applicant_data_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

Dropping the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [5]:
# Dropping the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN', 'NAME'])

# Reviewing the DataFrame
applicant_data_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


The dataset's categorical variables are encoded using `OneHotEncoder` and stored in a new DataFrame with original DataFrame's numerical variables.

In [6]:
# Creating a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == 'object'].index)

# Displaying the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
# Creating a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)


In [8]:
# Encoding the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [9]:
# Creating a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names(categorical_variables))

# Reviewing the DataFrame
encoded_df.head()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Adding the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat(
    [
        encoded_df,
        applicant_data_df.drop(columns=categorical_variables)
    ],axis=1)

# Reviewing the Dataframe
encoded_df.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590,1


Using the preprocessed data, the features(`X`) and target (`y`) dataframes are created.



In [11]:
# Defining the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Displaying a sample of y
y.head()


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [12]:
# Defining features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=['IS_SUCCESSFUL'])

# Reviewing the features DataFrame
X.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590


Training and testing datasets are created by splitting the features (`X`) and target (`y`).


In [13]:
# Splitting the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y)


Using `StandardScaler` to scale the features data.

In [14]:
# Creating a StandardScaler instance
scaler = StandardScaler()

# Fitting the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fitting the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

A binary classification deep neural network model is created using TensorFlow and Keras. The model uses the dataset's features to predict whether an Alphabet Soup funded startup will be successful.

We followed the below steps:

1. A deep neural network is created using Tensorflow's Keras by assigning the number of input features(`116`), the number of layers(`2`), and the number of neurons on each layer(`10 in layer 1 and 7 in layer 2`). ( We use `relu` as the activation function for the hidden layers)

2. The model is then compiled and fit using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric with `epochs=50`.

3. The model is evaluated using the test data to determine the model’s loss and accuracy.

4. The model is saved and exported to an HDF5 file named `AlphabetSoup.h5`.

Creating deep neural network


In [15]:
# Defining the the number of inputs (features) to the model
number_input_features = len(X_train.columns)

# Reviewing the number of features
number_input_features


116

In [16]:
# Defining the number of neurons in the output layer
number_output_neurons = 1

In [19]:
# Defining the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 10

# Reviewing the number hidden nodes in the first layer
hidden_nodes_layer1


10

In [18]:
# Defining the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  7

# Reviewing the number hidden nodes in the second layer
hidden_nodes_layer2


7

In [20]:
# Creating the Sequential model instance
nn = Sequential()


In [21]:
# Adding the first hidden layer
nn.add(Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))


In [22]:
# Adding the second hidden layer
nn.add(Dense(units = hidden_nodes_layer2, activation = 'relu'))


In [23]:
# Adding the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units = number_output_neurons, activation = 'sigmoid'))


In [24]:
# Display the Sequential model summary
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                1170      
                                                                 
 dense_1 (Dense)             (None, 7)                 77        
                                                                 
 dense_2 (Dense)             (None, 1)                 8         
                                                                 
Total params: 1,255
Trainable params: 1,255
Non-trainable params: 0
_________________________________________________________________


The model is then compiled and fit using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric with `epochs=50`.


In [25]:
# Compiling the Sequential model
nn.compile(loss= 'binary_crossentropy',
          metrics = ['accuracy'],
          optimizer = 'adam')


In [26]:
# Fitting the model using 50 epochs and the training data
fit_model_original = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 4ms/step - loss: 0.6098 - accuracy: 0.6999
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5647 - accuracy: 0.7241
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5594 - accuracy: 0.7267
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5568 - accuracy: 0.7267
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7279
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7283
Epoch 7/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5535 - accuracy: 0.7268
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5530 - accuracy: 0.7285
Epoch 9/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5523 - accuracy: 0.7282
Epoch 10/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5516 - accuracy: 0.7288

The model is evaluated using the test data to determine the model’s loss and accuracy.


In [27]:
# Evaluating the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Displaying the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5503 - accuracy: 0.7353 - 504ms/epoch - 2ms/step
Loss: 0.5502599477767944, Accuracy: 0.735276997089386


The model is saved and exported to an HDF5 file named `AlphabetSoup.h5`.


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
# Set the model's file path
file_path = 'Resources/AlphabetSoup.h5'

# Export your model to a HDF5 file
nn.save(file_path)



---

## Optimize the neural network model


The following models are created to optimize the original model's accuracy.



#### Alternative Model 1 - Adding Number of Epochs

* Nodes in Layer 1 - 10
* Nodes in Layer 2 - 7
* Number of Epochs - 300
* Hidden Layer Activation Function - relu

In [29]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [30]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [31]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = 10

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

10

In [32]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 = 7

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A1

7

In [33]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [34]:
# First hidden layer
nn_A1.add(Dense(units = hidden_nodes_layer1_A1, input_dim = number_input_features, activation = 'relu'))

#Second hidden layer
nn_A1.add(Dense(units = hidden_nodes_layer2_A1, activation = 'relu'))

# Output layer
nn_A1.add(Dense(units = number_output_neurons_A1, activation = 'sigmoid'))


# Check the structure of the model
nn_A1.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                1170      
                                                                 
 dense_4 (Dense)             (None, 7)                 77        
                                                                 
 dense_5 (Dense)             (None, 1)                 8         
                                                                 
Total params: 1,255
Trainable params: 1,255
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the Sequential model
nn_A1.compile(loss= 'binary_crossentropy',
             metrics = ['accuracy'],
             optimizer = 'adam')


In [36]:
# Fit the model using 200 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=300)


Epoch 1/300
804/804 [==============================] - 5s 4ms/step - loss: 0.6471 - accuracy: 0.6766
Epoch 2/300
804/804 [==============================] - 3s 3ms/step - loss: 0.5661 - accuracy: 0.7223
Epoch 3/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5589 - accuracy: 0.7237
Epoch 4/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7257
Epoch 5/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7264
Epoch 6/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7259
Epoch 7/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7265
Epoch 8/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7285
Epoch 9/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7280
Epoch 10/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accura

#### Alternative Model 2 - Adding Additional Layer (3 Layers)

* Nodes in Layer 1 - 10
* Nodes in Layer 2 - 7
* Nodes in Layer 3 - 5
* Number of Epochs - 75
* Hidden Layer Activation Function - relu

In [37]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [38]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [41]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = 10
# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

10

In [42]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 = 7
# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A2

7

In [43]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A2 = 5
# Review the number of hidden nodes in the first layer
hidden_nodes_layer3_A2

5

In [44]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [ ]:
# First hidden layer
nn_A2.add(Dense(units = hidden_nodes_layer1_A2, input_dim = number_input_features, activation = 'relu'))

# Second hidden layer
nn_A2.add(Dense(units = hidden_nodes_layer2_A2, activation = 'relu'))

# Third hidden layer
nn_A2.add(Dense(units = hidden_nodes_layer3_A2, activation = 'relu'))

# Output layer
nn_A2.add(Dense(units = number_output_neurons_A2, activation = 'sigmoid'))

# Check the structure of the model
nn_A2.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                1170      
                                                                 
 dense_7 (Dense)             (None, 7)                 77        
                                                                 
 dense_8 (Dense)             (None, 5)                 40        
                                                                 
 dense_9 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,293
Trainable params: 1,293
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Compile the model
nn_A2.compile(loss= 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'adam')


In [46]:
# Fit the model
fit_model_A2  = nn_A2.fit(X_train_scaled, y_train, epochs=75)

Epoch 1/75
804/804 [==============================] - 1s 1ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 2/75
804/804 [==============================] - 1s 1ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 3/75
804/804 [==============================] - 1s 1ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 4/75
804/804 [==============================] - 1s 2ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 5/75
804/804 [==============================] - 1s 1ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 6/75
804/804 [==============================] - 1s 1ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 7/75
804/804 [==============================] - 1s 1ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 8/75
804/804 [==============================] - 1s 1ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 9/75
804/804 [==============================] - 1s 1ms/step - loss: 7.8463 - accuracy: 0.0026
Epoch 10/75
804/804 [==============================] - 1s 2ms/step - loss: 7.8463 - accuracy: 0.0026

#### Alternative Model 3 - Adding more nodes to 2 layers
* Nodes in Layer 1 - 58
* Nodes in Layer 2 - 29
* Number of Epochs - 120
* Hidden Layer Activation Function - relu

In [47]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [48]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 1

In [49]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 = ((number_input_features + number_output_neurons_A3) // 2) 

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A3

58

In [50]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A3 = ((hidden_nodes_layer1_A3 + number_output_neurons_A3) // 2 ) 

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A3

29

In [51]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [52]:
# First hidden layer
nn_A3.add(Dense(units = hidden_nodes_layer1_A3, input_dim = number_input_features, activation = 'relu'))

# Second hidden layer
nn_A3.add(Dense(units = hidden_nodes_layer2_A3, activation = 'relu'))

# # Third hidden layer
# nn_A3.add(Dense(units = hidden_nodes_layer3_A3, activation = 'relu'))

# Output layer
nn_A3.add(Dense(units = number_output_neurons_A3, activation = 'sigmoid'))

# Check the structure of the model
nn_A3.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 58)                6786      
                                                                 
 dense_7 (Dense)             (None, 29)                1711      
                                                                 
 dense_8 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Compile the model
nn_A3.compile(loss= 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'adam')


In [54]:
# Fit the model
fit_model_A3 = nn_A3.fit(X_train_scaled, y_train, epochs=120)


Epoch 1/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5768 - accuracy: 0.7198
Epoch 2/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7287
Epoch 3/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7289
Epoch 4/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7299
Epoch 5/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7297
Epoch 6/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7310
Epoch 7/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7308
Epoch 8/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7325
Epoch 9/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5444 - accuracy: 0.7321
Epoch 10/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5444 - accura

#### Alternative Model 4 - Reducing input features using PCA
* Nodes in Layer 1 - 44
* Nodes in Layer 2 - 25
* Number of Epochs - 120
* Hidden Layer Activation Function - relu

In [55]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [56]:
pca_data_df = X.drop(columns=['ASK_AMT'])
scaled_data = StandardScaler().fit_transform(X[['ASK_AMT']])
scaled_data_df = pd.DataFrame(
    scaled_data,
    columns=["ASK_AMT"]
)
pca_scaled_data_df = pd.concat(
    [
        pca_data_df,
        scaled_data_df
    ],axis=1)
pca_scaled_data_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,-0.031725
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,-0.030536
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,-0.031725
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,-0.031706
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,-0.030146


In [57]:
# Creating a list with the number of k-values to try
k = list(range(5, 116, 5))

# Creating an empy list to store the inertia values
cumulative_explained_variance = []

In [58]:
# Creating a for loop to compute the cumulative explained variance with each possible value of k
for i in k:
    pca = PCA(n_components=i)
    applicant_pca_data = pca.fit_transform(pca_scaled_data_df)
    pca.explained_variance_ratio_
    cumulative_explained_variance.append(sum(pca.explained_variance_ratio_))


In [59]:
k_variance = {
    "k" : k,
    "cumulative_variance" : cumulative_explained_variance
}

# Create the DataFrame from the k_variance data
k_variance_df = pd.DataFrame(k_variance)

# Review the DataFrame
k_variance_df

,k,cumulative_variance
0,5,0.728984
1,10,0.891678
2,15,0.938370
3,20,0.966632
4,25,0.982110
5,30,0.988953
6,35,0.992999
7,40,0.995473
8,45,0.997035
9,50,0.998055


Since at k=100 the cumulative variance is 1. So the ideal value of `n_components` for PCA lies within the range of `95 to 105` 

In [60]:
# Creating a list with all number of k-values to try between 95 and 110
k = list(range(95, 111))

# Creating an empy list to store the cumulative explained variance
cumulative_explained_variance = []
# Instantiate the PCA instance
for i in k:
    pca = PCA(n_components=i)
    applicant_pca_data = pca.fit_transform(pca_scaled_data_df)
    pca.explained_variance_ratio_
    cumulative_explained_variance.append(sum(pca.explained_variance_ratio_))

k_variance = {
    "k" : k,
    "cumulative_variance" : cumulative_explained_variance
}

# Create the DataFrame from the k_variance data
k_variance_df = pd.DataFrame(k_variance)

# Review the DataFrame
k_variance_df

,k,cumulative_variance
0,95,0.999892
1,96,0.999900
2,97,0.999907
3,98,0.999915
4,99,0.999923
5,100,0.999931
6,101,0.999938
7,102,0.999946
8,103,0.999954
9,104,0.999962


Selecting k=95 for `n_components` for PCA

In [61]:
pca = PCA(n_components=95)
pca_result = pca.fit_transform(pca_scaled_data_df)
applicant_pca_data = pd.DataFrame(pca_result)
applicant_pca_data.head()

,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,94
0,-0.083188,0.765107,1.520044,-0.041563,0.417013,0.693788,-0.111376,-0.008989,0.160738,0.291843,...,3.150160e-16,-4.020319e-16,-2.258297e-15,9.225849e-16,2.072420e-15,-3.338874e-16,3.780070e-16,4.228024e-16,-1.613067e-15,7.719067e-15
1,0.036049,-0.487144,-0.068737,0.791803,-0.512586,0.570958,-0.737868,0.055880,0.038826,0.027265,...,7.388622e-16,-5.104545e-16,-3.637543e-15,1.532066e-15,3.139693e-15,-5.324045e-16,2.627643e-16,4.333147e-16,-2.492593e-15,1.353134e-14
2,-0.186581,1.895555,0.708600,0.627688,0.096104,-0.064978,0.321214,0.038125,-0.546968,-0.362178,...,9.703118e-17,3.693359e-17,-4.335002e-16,2.500169e-16,3.995623e-16,-9.210195e-17,9.153174e-17,4.661092e-17,-3.748418e-16,1.653262e-15
3,-0.021254,0.000487,-0.765658,0.509864,-0.623054,-0.752042,-0.514927,0.058387,0.170119,-0.183311,...,2.955181e-16,-2.214026e-16,-1.646207e-15,6.404256e-16,1.509893e-15,-2.370735e-16,1.425653e-16,3.110040e-16,-9.450823e-16,4.676339e-15
4,0.081041,-0.831770,0.825064,-0.092806,-0.543976,-0.201895,0.083927,0.747607,0.096142,0.053667,...,5.036988e-14,-4.076002e-14,-2.760453e-13,1.189549e-13,2.601088e-13,-4.145516e-14,3.323804e-14,4.626732e-14,-1.876257e-13,8.950160e-13


In [62]:
X_pca = applicant_pca_data.copy()
y_pca = y.copy()

In [63]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca,y_pca)

In [64]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_pca_train.iloc[0])

# Review the number of features
number_input_features

# Define the number of neurons in the output layer
number_output_neurons_A4 = 1

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A4 = ((number_input_features + number_output_neurons_A4) // 2) 

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A4

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A4 = ((hidden_nodes_layer1_A4+ number_output_neurons_A4) // 2 ) 

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A4

24

In [65]:
# Create the Sequential model instance
nn_A4 = Sequential()

# First hidden layer
nn_A4.add(Dense(units = hidden_nodes_layer1_A4, input_dim = number_input_features, activation = 'relu'))

# Second hidden layer
nn_A4.add(Dense(units = hidden_nodes_layer2_A4, activation = 'relu'))

# Output layer
nn_A4.add(Dense(units = number_output_neurons_A4, activation = 'sigmoid'))

# Check the structure of the model
nn_A4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 48)                4608      
                                                                 
 dense_10 (Dense)            (None, 24)                1176      
                                                                 
 dense_11 (Dense)            (None, 1)                 25        
                                                                 
Total params: 5,809
Trainable params: 5,809
Non-trainable params: 0
_________________________________________________________________


In [66]:
# Compile the model
nn_A4.compile(loss= 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

# Fit the model
fit_model_A4 = nn_A4.fit(X_pca_train, y_pca_train, epochs=120)

Epoch 1/120
804/804 [==============================] - 3s 3ms/step - loss: 0.5741 - accuracy: 0.7178
Epoch 2/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7327
Epoch 3/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7333
Epoch 4/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5443 - accuracy: 0.7330
Epoch 5/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5429 - accuracy: 0.7343
Epoch 6/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5420 - accuracy: 0.7345
Epoch 7/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5409 - accuracy: 0.7350
Epoch 8/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 0.7346
Epoch 9/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5400 - accuracy: 0.7362
Epoch 10/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5401 - accura

#### Alternative Model 5 - Changing Activation Function for hidden layer
* Nodes in Layer 1 - 58
* Nodes in Layer 2 - 29
* Number of Epochs - 120
* Hidden Layer Activation Function - tanh

In [67]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

# Define the number of neurons in the output layer
number_output_neurons_A5 = 1

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A5 = ((number_input_features + number_output_neurons_A5) // 2) 

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A5

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A5 = ((hidden_nodes_layer1_A5+ number_output_neurons_A5) // 2 ) 

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A5    

29

In [68]:
# Create the Sequential model instance
nn_A5 = Sequential()

# First hidden layer
nn_A5.add(Dense(units = hidden_nodes_layer1_A5, input_dim = number_input_features, activation = 'tanh'))

# Second hidden layer
nn_A5.add(Dense(units = hidden_nodes_layer2_A5, activation = 'tanh'))

# Output layer
nn_A5.add(Dense(units = number_output_neurons_A5, activation = 'sigmoid'))

# Check the structure of the model
nn_A5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 58)                6786      
                                                                 
 dense_13 (Dense)            (None, 29)                1711      
                                                                 
 dense_14 (Dense)            (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Compile the model
nn_A5.compile(loss= 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

# Fit the model
fit_model_A5 = nn_A5.fit(X_train_scaled, y_train, epochs=120)

Epoch 1/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5768 - accuracy: 0.7126
Epoch 2/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5568 - accuracy: 0.7284
Epoch 3/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7296
Epoch 4/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7285
Epoch 5/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7308
Epoch 6/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7333
Epoch 7/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5457 - accuracy: 0.7334
Epoch 8/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accuracy: 0.7329
Epoch 9/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accuracy: 0.7313
Epoch 10/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5432 - accura

### Displaying the accuracy scores achieved by each model, and comparing the results.

In [70]:
print("Original Model Results")
print("Nodes in Layer 1 - 10\nNodes in Layer 2 - 7\nNumber of Epochs - 50\nHidden Layer Activation Function - relu")

# Evaluating the model loss and accuracy metrics
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Displaying the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
Nodes in Layer 1 - 10
Nodes in Layer 2 - 7
Number of Epochs - 50
Hidden Layer Activation Function - relu
268/268 - 0s - loss: 0.5503 - accuracy: 0.7353 - 387ms/epoch - 1ms/step
Loss: 0.5502599477767944, Accuracy: 0.735276997089386


In [71]:
print("Alternative Model 1 Results - Adding more Epochs (300)")
print("Nodes in Layer 1 - 10\nNodes in Layer 2 - 7\nNumber of Epochs - 300\nHidden Layer Activation Function - relu")

# Evaluating the model loss and accuracy metrics
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled,y_test,verbose=2)

# Displaying the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results - Adding more Epochs (300)
Nodes in Layer 1 - 10
Nodes in Layer 2 - 7
Number of Epochs - 300
Hidden Layer Activation Function - relu
268/268 - 0s - loss: 0.5561 - accuracy: 0.7380 - 457ms/epoch - 2ms/step
Loss: 0.5560914874076843, Accuracy: 0.7379592061042786


In [72]:
print("Alternative Model 2 Results - Adding more layers")
print("Nodes in Layer 1 - 10\nNodes in Layer 2 - 7\nNodes in Layer 2 - 5\nNumber of Epochs - 75\nHidden Layer Activation Function - relu")

# Evaluating the model loss and accuracy metrics
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled,y_test,verbose=2)

# Displaying the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results - Adding more layers
Nodes in Layer 1 - 10
Nodes in Layer 2 - 7
Nodes in Layer 2 - 5
Number of Epochs - 75
Hidden Layer Activation Function - relu
268/268 - 0s - loss: 7.8659 - accuracy: 0.0024 - 425ms/epoch - 2ms/step
Loss: 7.865939140319824, Accuracy: 0.0024489795323461294


In [73]:
print("Alternative Model 3 Results - Adding more nodes to 2 layers")
print("Nodes in Layer 1 - 58\nNodes in Layer 2 - 29\nNumber of Epochs - 120\nHidden Layer Activation Function - relu")

# Evaluating the model loss and accuracy metrics
model_loss, model_accuracy = nn_A3.evaluate(X_test_scaled,y_test,verbose=2)

# Displaying the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results - Adding more nodes to 2 layers
Nodes in Layer 1 - 58
Nodes in Layer 2 - 29
Number of Epochs - 120
Hidden Layer Activation Function - relu
268/268 - 0s - loss: 0.5597 - accuracy: 0.7374 - 460ms/epoch - 2ms/step
Loss: 0.559668242931366, Accuracy: 0.7373760938644409


In [74]:
print("Alternative Model 4 Results - Reduce Features using PCA")
print("Nodes in Layer 1 - 44\nNodes in Layer 2 - 25\nNumber of Epochs - 120\nHidden Layer Activation Function - relu")

# Evaluating the model loss and accuracy metrics
model_loss, model_accuracy = nn_A4.evaluate(X_pca_test,y_pca_test,verbose=2)

# Displaying the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



Alternative Model 4 Results - Reduce Features using PCA
Nodes in Layer 1 - 44
Nodes in Layer 2 - 25
Number of Epochs - 120
Hidden Layer Activation Function - relu
268/268 - 1s - loss: 0.5651 - accuracy: 0.7310 - 519ms/epoch - 2ms/step
Loss: 0.5650659799575806, Accuracy: 0.7309620976448059


In [75]:
print("Alternative Model 5 Results - Change Activation function")
print("Nodes in Layer 1 - 58\nNodes in Layer 2 - 29\nNumber of Epochs - 120\nHidden Layer Activation Function - tanh")
# Evaluating the model loss and accuracy metrics
model_loss, model_accuracy = nn_A5.evaluate(X_test_scaled,y_test,verbose=2)

# Displaying the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 5 Results - Change Activation function
Nodes in Layer 1 - 58
Nodes in Layer 2 - 29
Number of Epochs - 120
Hidden Layer Activation Function - tanh
268/268 - 0s - loss: 0.5532 - accuracy: 0.7384 - 458ms/epoch - 2ms/step
Loss: 0.5532391667366028, Accuracy: 0.7384256720542908


### Saving each alternative models as an HDF5 file.


In [76]:
# Setting the file path for the first model 1
file_path = 'Resources/AlphabetSoup_A1.h5'

# Exporting model to a HDF5 file
nn_A1.save(file_path)


In [77]:
# Setting the file path for the alternative model 2
file_path = 'Resources/AlphabetSoup_A2.h5'

# Exporting model to a HDF5 file
nn_A2.save(file_path)


In [78]:
# Setting the file path for the alternative model 3
file_path = 'Resources/AlphabetSoup_A3.h5'

# Exporting model to a HDF5 file
nn_A3.save(file_path)

In [79]:
# Setting the file path for the alternative model 4
file_path = 'Resources/AlphabetSoup_A4.h5'

# Exportting model to a HDF5 file
nn_A4.save(file_path)

In [80]:
# Setting the file path for the alternative model 5
file_path = 'Resources/AlphabetSoup_A5.h5'

# Exporting model to a HDF5 file
nn_A5.save(file_path)